## Setup

In [1]:
import numpy as np
import numcodecs
numcodecs.__version__

'0.4.2.dev2+dirty'

In [2]:
import fastparquet


class FastParquetCodec(numcodecs.abc.Codec):
    """Hacked codec using fastparquet utf8 encoding, for benchmarking purposes only."""
    
    codec_id = 'xxx-fastparquet'
    
    def encode(self, buf):
        buf = np.asanyarray(buf)
        n = buf.size
        ba = fastparquet.speedups.array_encode_utf8(buf)
        enc = fastparquet.speedups.pack_byte_array(ba.tolist())
        return n, enc  # hack for now, return n
    
    def decode(self, data, out=None):
        n, enc = data
        ba = fastparquet.speedups.unpack_byte_array(enc, n)
        dec = fastparquet.speedups.array_decode_utf8(np.array(ba, dtype=object))
        if out is not None:
            out[:] = dec
            return out
        return dec

In [3]:
zstd1 = numcodecs.Zstd(1)
zstd5 = numcodecs.Zstd(5)
zstd9 = numcodecs.Zstd(9)


def benchmark_codec(codec, a):
    print(codec)
    print('encode')
    %timeit codec.encode(a)
    enc = codec.encode(a)
    print('decode')
    %timeit codec.decode(enc)
    if isinstance(codec, FastParquetCodec):
        enc = enc[1]  # hack
    print('size         : {:,}'.format(len(enc)))
    print('size (zstd 1): {:,}'.format(len(zstd1.encode(enc))))
    print('size (zstd 5): {:,}'.format(len(zstd5.encode(enc))))
    print('size (zstd 9): {:,}'.format(len(zstd9.encode(enc))))


In [4]:
from numcodecs.tests.common import greetings
msgpack_codec = numcodecs.MsgPack()
json_codec = numcodecs.JSON()
pickle_codec = numcodecs.Pickle()
cat_codec = numcodecs.Categorize(greetings, dtype=object, astype='u1')
vlen_codec = numcodecs.VLenUTF8()
fp_codec = FastParquetCodec()

## Greetings benchmark

In [5]:
np.random.seed(42)
data = np.random.choice(greetings, size=1000000).astype(object)
data

array(['Γεια σου κόσμε!', 'Hei maailma!', 'Zdravo svete!', ...,
       'Servus Woid!', 'เฮลโลเวิลด์', 'Zdravo svete!'], dtype=object)

In [6]:
benchmark_codec(msgpack_codec, data)

MsgPack(encoding='utf-8')
encode
127 ms ± 9.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
225 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
size         : 18,996,503
size (zstd 1): 1,576,435
size (zstd 5): 1,409,320
size (zstd 9): 1,310,380


In [7]:
benchmark_codec(json_codec, data)

JSON(encoding='utf-8', allow_nan=True, check_circular=True, ensure_ascii=True,
     indent=None, separators=(',', ':'), skipkeys=False, sort_keys=True,
     strict=True)
encode
184 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
decode
469 ms ± 44.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
size         : 33,322,595
size (zstd 1): 1,840,414
size (zstd 5): 1,675,163
size (zstd 9): 1,522,853


In [8]:
benchmark_codec(pickle_codec, data)

Pickle(protocol=4)
encode
304 ms ± 58.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
decode
225 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
size         : 20,835,275
size (zstd 1): 1,608,227
size (zstd 5): 1,436,093
size (zstd 9): 1,333,676


In [9]:
benchmark_codec(cat_codec, data)

Categorize(dtype='|O', astype='|u1', labels=['¡Hola mundo!', 'Hej Världen!', 'Servus Woid!', ...])
encode
286 ms ± 58.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
decode
36.7 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 1,000,000
size (zstd 1): 458,191
size (zstd 5): 493,638
size (zstd 9): 490,483


In [6]:
benchmark_codec(vlen_codec, data)

VLenUTF8()
encode
21.9 ms ± 985 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
161 ms ± 3.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 21,830,287
size (zstd 1): 1,762,831
size (zstd 5): 1,546,643
size (zstd 9): 1,358,821


In [11]:
benchmark_codec(fp_codec, data)

FastParquetCodec()
encode
89 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
222 ms ± 4.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
size         : 21,830,271
size (zstd 1): 1,762,809
size (zstd 5): 1,546,612
size (zstd 9): 1,358,813


## Lorem benchmark

In [13]:
from faker import Faker
fake = Faker()

In [14]:
data2 = np.array(' '.join(fake.sentences(nb=200000)).split(), dtype=object)
len(data2), data2[:10]

(1101344, array(['Magnam', 'ipsam', 'ad.', 'Molestias', 'tempore', 'ipsa',
        'corporis', 'nisi.', 'Exercitationem', 'alias'], dtype=object))

In [13]:
benchmark_codec(msgpack_codec, data2)

MsgPack(encoding='utf-8')
encode
105 ms ± 7.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
170 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 8,380,619
size (zstd 1): 2,631,293
size (zstd 5): 2,358,494
size (zstd 9): 2,265,121


In [14]:
benchmark_codec(json_codec, data2)

JSON(encoding='utf-8', allow_nan=True, check_circular=True, ensure_ascii=True,
     indent=None, separators=(',', ':'), skipkeys=False, sort_keys=True,
     strict=True)
encode
121 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
141 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 10,584,987
size (zstd 1): 2,547,087
size (zstd 5): 2,438,584
size (zstd 9): 2,312,637


In [15]:
benchmark_codec(pickle_codec, data2)

Pickle(protocol=4)
encode
242 ms ± 7.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
decode
138 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 10,593,806
size (zstd 1): 2,584,223
size (zstd 5): 2,456,961
size (zstd 9): 2,455,358


In [15]:
benchmark_codec(vlen_codec, data2)

VLenUTF8()
encode
20.1 ms ± 844 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
78.3 ms ± 693 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 11,680,886
size (zstd 1): 2,956,544
size (zstd 5): 2,860,531
size (zstd 9): 2,556,631


In [16]:
benchmark_codec(fp_codec, data2)

FastParquetCodec()
encode
80.6 ms ± 3.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
decode
142 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
size         : 11,680,870
size (zstd 1): 2,956,599
size (zstd 5): 2,860,473
size (zstd 9): 2,556,623
